# 自动批量生成物料标签 <span class="graffiti-highlight graffiti-id_gz13ph1-id_9uzfhnw"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>


## 实验目的：

- 掌握excel文件读取方法 
- 掌握信息提取的方法
- 掌握正则表达式

## 实验内容

- 介绍openpyxl库以及使用方法 
- 编写自定义函数
- 使用正则表达式匹配信息

按照客户要求，每种产品在出货时都需要在包装上贴上如下信息标签，以便识别。

<img style="float: left;" width = "350" height = "300" src=./images/prod_label.PNG>

标签上需要包含6种信息，原始资料内容如下：

<img style="float: left;" width = "500" height = "300" src=./images/prod_info.png>

我们注意到原始信息中的品牌，有些是大写，有些是小写，而标签要求全部大写，因此在填入之前需要先做处理。公司的常规做法是充分发挥人海战术，安排一个小组的人一哄而上，先手动将小写字母替换为大写，然后开始一系列的"复制"加"粘贴"。通常完成一个物料的标签信息需要35秒，100个物料就需要约1小时。可怜那些操作人员，除了手酸，脖子硬，眼睛疼，再加上键盘上的“ctrl，C，V”三个键像光头一样光滑外，还容易出错。如果原始信息有误，部分标签还得小心翼翼、如履薄冰地重新复制粘贴来修正。有时候，操作人员一走神，复制或者粘贴错了，还会被客户投诉，继而被老板训斥，整个心情都不好了。

通常一个订单有上百种物料，每天需要处理约30个订单，需要4个人全职处理这些信息。使用Python, 只需要很少量代码即可轻松搞定，运行只需几秒钟时间。只要原始信息没问题，Python会保质保量完成任务。

# <font color='royalblue'> 1. 提取和规范数据 </font> 
首先，我们需要将空白标签的格式复制足够多个。为方便打印，本例是在A4纸上每排放3个标签。也可让Python去自动填写标签第一列的信息，但涉及格式操作，还不如在Excel内直接设置方便。



以上操作完成后，就可以开始后续的信息读取和填写了，步骤如下：
> 1. 从“info”工作表读取所有物料的信息
> 2. 转换字母为大写，存储好信息
> 3. 将处理好的信息逐个写入到“label”工作表
> 4. 保存工作表



## 1.1 安装excel编辑库  

In [ ]:
!pip install openpyxl

## 1.2 提取和整理表格中的信息 

<img style="float: left;" width = "450" height = "300" src=./images/01.png>

<img style="float: left;" width = "450" height = "300" src=./images/02.png>

### 1.2.1 获取特定单元格的信息   <span class="graffiti-highlight graffiti-id_vvqkjs9-id_3bttjli"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span> 
我们首先导入`openpyxl`模块中的`load_workbook`模块。`openpyxl`是一个可读取Excel的第三方库。`load_workbook()`可打开一个现有的Excel文件。
我们使用`load_workbook（path）`读取文件，并存在变量`wb`里面。我们可以想象变量就是容器，用来储存各种数据，这里的`wb`就是这个容器的名字。

点击下载[label_info.xlsx](../../notebooks/generate-labels_and_re/data/label_info.xlsx)

In [ ]:
from openpyxl import load_workbook

wb = load_workbook('data/label_info.xlsx')  # 让我们先加载一个excel表格

但是我们并不需要整个Excel文件的内容，只需要其中一个名叫“info”的工作表的内容，因此我们再定义一个变量（容器）`ws`从容器`wb`中将工作表“info”中的数据取过来并放进去：`ws = wb['info']`。`ws`装下了工作表“info”中的内容。

In [ ]:
ws = wb['info']    # 声明名为‘info’的工作表

# 获取某一单元格数值的方法

a = ws['A16'].value  # 获取单元格A16的值
b = ws['C2'].value # 获取单元格C2的值
print(a)
print(b)

### <font color='red'>练习:根据代码注释填空<font> <span class="graffiti-highlight graffiti-id_k1nsaec-id_ug78q7g"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

In [ ]:
#获取E2单元格的信息
c = ws['E2'].value
print(c)

`ws.max_row`可获得这个工作表的行数。比如本例的“info”工作表有71行数据，那这个`ws.max_row`就是71。
`ws.max_column` 可获得这个工作表的列数。<span class="graffiti-highlight graffiti-id_1x9hjes-id_zbw5dm0"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

In [ ]:
# 某一列的行数最大的数值的方法
max_row = ws.max_row
print(type(max_row))

print(max_row)
d = ws['A' + str(max_row)].value
print(d)


### <font color='red'>练习:根据代码注释填空<font> <span class="graffiti-highlight graffiti-id_mz7slf1-id_dqznbx5"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

In [ ]:
# 获取最大列数
max_column = ws.max_column
print(max_column)


### 1.2.3 转换大小写的方法  <span class="graffiti-highlight graffiti-id_qktkbdz-id_r56hm26"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

`upper()`是转换成大写，
`lower()`是转换成小写

In [ ]:
# 将文本转换为大写
text1 = 'aBc333'
text2 = 'abc123'
text3 = 'ABC35'

text1 = text1.upper()
text2 = text2.upper()
text3 = text3.upper()
print(text1,text2,text3)


### <font color='red'>练习:根据代码注释填空<font> <span class="graffiti-highlight graffiti-id_hueau7j-id_679o3jq"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

In [ ]:
# 将sentence里面的文字转换为小写
sentence = 'I LIKE Apples.'
sentence = ??
print(sentence)

### 1.2.4 截取日期的方法 <span class="graffiti-highlight graffiti-id_fsz6k9b-id_1klpa1x"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

In [ ]:
# 获取日期
date1 = ws['E3'].value
print(date1)

# 省略后面的具体时间，只需要日期
date1 = date1.date()
print(date1)

### 1.2.5 遍历文档的方法并添加到列表  <span class="graffiti-highlight graffiti-id_p51e12z-id_nkalldb"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

由于从`range`函数中取值时，是不包含末尾那个数的，所以此处需要`max_row` + 1= 72，才能最多取到第71行的数据。

In [ ]:
for i in range(2,5):# 从2-5范围内逐个取值，但不包含5
    print(i)

### 遍历A列的所有数据，并将它们全部改成大写，然后把他们添加到列表中 

In [ ]:
# 遍历A列的所有数据，并将它们全部改成大写，然后他们添加到列表中
max_row = ws.max_row
info = []

for row in range(2, max_row + 1):
    brand = ws['A' + str(row)].value
    if brand:               # 判断商标这格是否为空，不为空的话改成大写并加入列表
        brand = brand.upper()
        info.append(brand)
    
print(info)


### 1.2.6 创建字典添加文档元素 <span class="graffiti-highlight graffiti-id_s746i3w-id_agov7s5"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>
<img style="float: left;" width = "250" height = "300" src=./images/prod_label.PNG>

In [ ]:
# 创建一个字典，用来保存一件（任意一行）商品的所有信息

row = 2  # 第2行

brand = ws['A' + str(row)].value
typ = ws['B' + str(row)].value
pn = ws['C' + str(row)].value
location = ws['D' + str(row)].value
date = ws['E' + str(row)].value.date()
quantity = str(ws['F' + str(row)].value)+ 'pcs'  # 在数量后面加上“pcs”字样

# key 键

data={
            "brand":brand,
            "typ":typ,
            "pn":pn,
            "lotno":location,
            "date":date,
            "quantity":quantity
        }

print(data)

### <font color='red'>练习:根据代码注释填空<font> <span class="graffiti-highlight graffiti-id_psxz39y-id_qf2ods1"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

In [ ]:
# 创建一个字典，用来保存一件（任意一行）商品的所有信息
# 这次来第5行的所有信息




### 字典的键值应用  

In [ ]:
keys = list(data.keys())
print(keys)
print(data[keys[0]])
print(data['brand'])





### 1.2.7 列表和字典的合并使用

### <font color='red'>练习:根据代码注释填空<font> <span class="graffiti-highlight graffiti-id_ppmn8gx-id_1pyqxto"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

In [ ]:
# 也可以创建一个列表来储存字典
info = []
# 将刚刚的两个字典都加入到info列表中
info.append(data)

print(info)


# <font color='red'>2. 根据提示补全下方函数<font> <span class="graffiti-highlight graffiti-id_8bfvev7-id_uttdk7e"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

此处我们先定义一个函数，取名为`get_info`,该函数包含一个参数`path`,用来传入所需要打开的Excel文件的路径。<br>
定义函数是为了方便后续反复地调用，因为我们后续需要处理的文件可能在不同的路径，我们只需传入不同的文件路径即可获取该文件的信息，而不用每次都去函数内部做改动。



我们建立了一个空列表`info`来存储从`ws`中按每种物料的信息取出来的数据。然后从第二行开始（第一行是列名）通过`for`循环来逐个查看并获取每行的数据，存进一个字典`data`，再将`data`放入`info`列表中。

函数名为`get_info(path)`
> 参数和性质：
  - `path`: 字符串类型，需要获得信息的excel文件的地址
  - `return`: 字符串类型，翻译的结果


In [ ]:
# 从“info”工作表读取并处理信息，存入列表
from openpyxl import load_workbook
def get_info(path):
    wb = load_workbook(path)
    ws = wb['info']
    info=[]
    for row in range(2, ws.max_row+1):
        brand = ws['A' + str(row)].value
        if brand:
            brand=brand.upper() #将“品牌”中的小写字母全部转换成大写字母
        typ = ws['B' + str(row)].value # 获取单元格中的数据
        pn = ws['C' + str(row)].value
        lotno = ws['D' + str(row)].value
        date = ws['E' + str(row)].value
        if date:
            date=date.date() #只获取日期时间中的日期，比如2019-2-20，不需要具体时间
        quantity = str(ws['F' + str(row)].value)+" pcs" # 在数量后面加上“pcs”字样
        data={
            "brand":brand,
            "typ":typ,
            "pn":pn,
            "lotno":lotno,
            "date":date,
            "quantity":quantity
        }
        info.append(data)
    return info
        
        
        
   

## <font color='red'> 2.1 检查下方代码是否能正确运行 　</font> <span class="graffiti-highlight graffiti-id_4b5v3hm-id_gwf59fb"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>
下方代码如能正确运行，说明没有编译错误，否则请继续修改你的函数。 
[label_info.xlsx](../../notebooks/generate-labels_and_re/data/label_info.xlsx)

In [ ]:
import os

In [ ]:
%%time
path = "data/label_info.xlsx"
info = get_info(path)
print("finished")

## <font color='red'> 2.2 根据说明检查运行结果

数据提取完成后，我们就可以检查我们的数据是否有问题。`len(info)`可查看info列表中有多少个数据，此处是70个数据，正好与Excel中的数据行数相同。再看info列表中的第一个数据`info[0]`（注意列表的索引是从0开始，0就表示第一个数据，以此类推）：
```python
{'brand': 'ABC1',
 'typ': '11X11-XX761X8',
 'pn': '011-0076108',
 'lotno': 'Q19H023994901',
 'date': datetime.date(2019, 2, 20),
 'quantity': '300 pcs'}
```
字典里的键对应的是“品牌、型号、物料编号、批次号、生成日期、数量”，值正好是Excel表中第2行的数据。

In [ ]:
print(len(info))

print(info[45])
print(info)

## <font color='red'> 2.3 Debug:检查空值 <span class="graffiti-highlight graffiti-id_wqsgxkp-id_o6wdklv"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

在`1.xlsx`故意放了数行含有空值的数据，可以看到倒数第二个数据的“品牌”和“生产日期”是“None”，即为空值。<br>
而空值就无法执行像`upper()`，`date()`这两个函数的方法了。<br>
点击下载[1.xlsx](../../notebooks/generate-labels_and_re/data/1.xlsx)

### <font color='red'>如果你写的函数出现了bug，请回到2.1修改你的函数，根据1.2.5，仿照if条件语句，设置判断控制的代码，解决bug问题。

<img style="float: left;" width = "450" height = "300" src=./images/3.png>

In [ ]:
info_nan=get_info("data/1.xlsx")

<font color='red'>此处为参考答案，双击后查看 </font> <span class="graffiti-highlight graffiti-id_13571x5-id_2udq291"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>
<p hidden>
def get_info(path):
    wb = load_workbook(path)
    ws = wb['info']
    info=[]
    for row in range(2, ws.max_row+1):
        brand = ws['A' + str(row)].value
        if brand:
            brand=brand.upper() #将“品牌”中的小写字母全部转换成大写字母
        typ = ws['B' + str(row)].value # 获取单元格中的数据
        pn = ws['C' + str(row)].value
        lotno = ws['D' + str(row)].value
        date = ws['E' + str(row)].value
        if date:
            date=date.date() #只获取日期时间中的日期，比如2019-2-20，不需要具体时间
        quantity = str(ws['F' + str(row)].value)+" pcs" # 在数量后面加上“pcs”字样
        data={
            "brand":brand,
            "typ":typ,
            "pn":pn,
            "lotno":lotno,
            "date":date,
            "quantity":quantity
        }
        info.append(data)
    return info
</p>

# <font color='royalblue'> 3. 将数据写入excel表格  <span class="graffiti-highlight graffiti-id_j838qfd-id_l56wikw"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

我们成功获取了所有产品的数据，并存进了列表容器`info`中。下一步就需要将其中的信息逐个提取出来，并写入到标签工作表里去了。<br>
我们的目的是将`label_info.xlsx`工作簿中的`info`中的信息填写到`label`中去，所以此处还是在`label_info.xlsx`中操作。

## 3.1 跳跃遍历 <span class="graffiti-highlight graffiti-id_ksbiye7-id_znmpdyo"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

`for`循环遍历所有列。为方便打印，我们的标签是3个一行，每个标签之间空一行一列。而我们只给第二、五、八列填写数据，因此这里的`range(2,9,3)`指的是从2列开始，8列结束，以3为步长，列的取值将为`2,5,8`，正是我们需要填写数据的列号。
<img style="float: left;" width = "450" height = "300" src=./images/04.png>

### 3.1.1 列遍历

In [ ]:
for i in range(2,9,3):   # 2,5,8 列需要写入我们的数据
    print(i)         # 此处代表从2到8，步长为3
    

### 3.1.2 行遍历 

随后我们进行行遍历，由于每个标签有6个信息，加上1个空行，因此每个标签总共占用了7行。<br>
`range(1,round(len(info)*7/3),7)`指从第一行开始，以步长为7取值，直到最后一行,`round`用于四舍五入以保证为整数，此处取的值都是每个标签的第一行。假设共有6个标签需要填写，那么需要的行数为`6*7/3=14`，`range(1,14,7)`可取到的值为1，8行。

In [ ]:
print(len(info))

In [ ]:
for j in range(1,round(len(info)*7/3),7):  # 1，8，15，22...... 是我们要写入数据的起始行
    print(j)

In [ ]:
for j in range(1,round(len(info)*7/3),7):  # 1，8，15，22...... 是我们要写入数据的起始行
    print(j)

### 3.1.3 行列结合  <span class="graffiti-highlight graffiti-id_gt3sdeq-id_rylbvh3"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>
现在我们需要在特定的行列单元格内遍历并写入数据，如下图所示。我们需要将上面的知识结合起来使用。

<img style="float: left;" width = "700" height = "300" src=./images/06.png>

In [ ]:
for i in range(2,9,3): # 列
    for j in range(1,round(len(info)*7/3),7): #行
        print(i,j) # 打印特定的行列

## 3.2 写入数据 

先用`load_workbook`打开Excel表，然后用`wb`这个容器来存储其所有信息。然后用`ws`容器存储`label`这张工作表里的信息。

### 3.2.1  在特定单元格写入数据

`test.xlsx` 的label工作表是空白的，下面的代码可以修改其第2行，第4列的单元格的数值。<br>
点击下载[test.xlsx](../../notebooks/generate-labels_and_re/data/test.xlsx)

In [ ]:
from excel import clean_all   
clean_all('data/test.xlsx','label')  # 清理excel文件里的多余信息，方便学生观察变化

In [ ]:
wb = load_workbook('data/test.xlsx')
ws = wb['label']
ws.cell(row=2,column=4).value = 'Hello'   # 在第2行第4列
wb.save('data/test.xlsx')

### <font color='red'>练习:根据代码注释填空<font> <span class="graffiti-highlight graffiti-id_vdo3g33-id_61hn2vl"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

In [ ]:
# 在5行6列写入任意数据

ws.cell(?) = ?
wb.?

### 3.2.2 遍历修改单元格  <span class="graffiti-highlight graffiti-id_klw912j-id_qyczadm"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>
我们定义了一个变量k,并设定它的初始值为0。因为数据可能不是3的整数倍，此时计数器`k`可能会超出列表`Info`的范围导致程序报错而终止，所以加上`if`语句，后续代码只有在<br>
`k <len(info)`的情况下才执行。

然后就是写入数据了。以第一个标签为例，我们在第一行第二列写入第一个物料的“品牌”信息，第二行第二列写入第一个物料的“型号”信息，一次类推。每写完一个物料的标签信息，计数器k需要加上一个1。因为`k`是列表`info`的索引，我们写好了`info[0]`里面的信息后，就需要接着写`info[1]`的信息了。
运行下方单元格会得到以下结果。

<img style="float: left;" width = "600" height = "300" src=./images/07.png>

In [ ]:
from excel import clean_all   
clean_all('data/test.xlsx','label')  # 清理excel文件里的多余信息，方便学生观察变化

### <font color='red'>练习:根据代码注释填空<font> <span class="graffiti-highlight graffiti-id_jlztf8n-id_xfbowpk"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

<span class="graffiti-highlight graffiti-id_x7noaqp-id_5ud9f3f"><i></i><button>观看讲解</button></span>

In [ ]:
from openpyxl import load_workbook
wb = load_workbook('data/test.xlsx')
ws = wb['label']

k=0 # 遍历info的索引
for i in range(2,9,3): # 列遍历
    for j in range(1,round(len(info)*7/3),7): # 行遍历
        if k < len(info): # 让计数器k不会超过列表info的长度范围
            ws.cell(row=j, column=i).value = info[k]['brand']
            ws.cell(row=j+1, column=i).value = info[k]['typ']
            ws.cell(row=j+2, column=i).value = info[k]['pn']
            ws.cell(row=j+3, column=i).value = info[k]['lotno']
            ws.cell(row=j+4, column=i).value = info[k]['date']
            ws.cell(row=j+5, column=i).value = info[k]['quantity']
        k = k + 1 # k是列表info的索引，此处加一以便获取下一条数据
        
wb.save('data/test.xlsx') # 保存打开的excel文件

# <font color='red'>4. 根据提示补全下方函数<font> <span class="graffiti-highlight graffiti-id_3oie1gb-id_ygyphex"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>
第二个函数`write_info()`，它包含两个参数，`path`及`info`,即需操作的文件及装着信息的容器。这个函数的功能是将容器里的信息写入到需要操作的文件，并保存，不需要返回什么值，因此没有`return`语句。

填入参数后运行完函数得到的结果如下图：

<img style="float: left;" width = "600" height = "300" src=./images/08.png>

In [ ]:
# 将处理好的信息逐个写入到“label”工作表并保存
def write_info(path,info):
    wb = load_workbook(path)
    ws = wb['label']
    
    # 列遍历
        #行遍历
        #当数据条数不是3的整数倍时，计数器k会超出列表info的范围，后续代码只有在k < len(info)的情况下执行
        
    
    
    
    
    
    
    #k是列表info的索引，此处加一以便获取下一条数据
    #保存Excel文件

## <font color = 'red'>4.1 检查下方代码是否能正确运行 <span class="graffiti-highlight graffiti-id_jxkcgvq-id_w96cf0r"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>
下方代码如能正确运行，说明没有编译错误，否则请继续修改你的函数。

In [ ]:
%%time
info=get_info("data/test.xlsx")
path="data/test.xlsx"
write_info(path,info) #调用write_info函数，将数据写入并保存
print("Finished!")

## <font color='red'> 4.2 根据说明检查运行结果</font>
请到`/data`文件夹下，下载[test.xlsx](../../notebooks/generate-labels_and_re/data/test.xlsx)文件，检查第190行开始的数据是否与下方的一致，如果一致说明函数正确。<br>
如果你的表格有错请运行下方的单元格清楚数据重新修改再检测

In [ ]:
from excel import clean_all   
clean_all('data/test.xlsx','label')  # 清理excel文件里的多余信息，方便学生

<img style="float: left;" width = "600" height = "300" src=./images/09.png>

<font color='red'> 此处为参考答案，双击后查看 </font> <span class="graffiti-highlight graffiti-id_7hwn90a-id_898rjkh"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>
<p hidden>
    '''
    def write_info1(path,info):
    wb = load_workbook(path)
    ws = wb['label']
    k=0
    for i in range(2,9,3): #列遍历
        for j in range(1,round(len(info)*7/3),7):
            if k<len(info):
                ws.cell(row=j, column=i).value = info[k]['brand']
                ws.cell(row=j, column=i-1).value = '品牌 \nBrand'

                ws.cell(row=j + 1, column=i).value = info[k]['typ']
                ws.cell(row=j + 1,column=i-1).value = '型号 Type'

                ws.cell(row=j + 2, column=i).value = info[k]['pn']
                ws.cell(row=j + 2,column=i-1).value = '物料编号 Item P/N'

                ws.cell(row=j + 3, column=i).value = info[k]['lotno']
                ws.cell(row=j + 3,column=i-1).value = '生产批号 Lot No.'

                ws.cell(row=j + 4, column=i).value = info[k]['date']
                ws.cell(row=j + 4,column=i-1).value = '生产日期 Date'

                ws.cell(row=j + 5, column=i).value = info[k]['quantity']
                ws.cell(row=j + 5,column=i-1).value = '数量 Quantity'
            k+=1 
    wb.save(path)
                   
    '''                       
</p>

# <font color='royalblue'> 5. 给数据加上边框 <span class="graffiti-highlight graffiti-id_55d9qj4-id_s0bmvkg"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>
现在我们需要给有数据的单元格加上边框，如图所示：

 
<img style="float: left;" width = "600" height = "300" src=./images/10.png>  

## 5.1 给特定单元格加上边框

In [ ]:
from excel import clean_all   
clean_all('data/test.xlsx','label')  # 清理excel文件里的多余信息，方便学生观察变化

In [ ]:
from openpyxl.styles.borders import Border, Side　　# 导入包
from openpyxl import Workbook       


# 创建一个border的类，并且定义它的特征
thin_border = Border(left=Side(style='thin'), 　　# 左边框为thin，细边框
                     right=Side(style='thin'),   # 右边框
                     top=Side(style='thin'),     # 上边框
                     bottom=Side(style='thin'))  # 下边框

path="data/test.xlsx     
wb = load_workbook(path)   # 加载excel文件
ws = wb['info']  # 打开info工作表
ws.cell(row= 1, column=8).border = thin_border  # 将第1行第8列的单元格应用成我们刚创建的border
wb.save(path)    # 保存加载的excel文件




# <font color='red'>6. 根据提示补全下方函数<font> <span class="graffiti-highlight graffiti-id_elc4onv-id_bptbvdb"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

现在我们需要创建一个函数`add_border()`，它的输入参数是excel的文件地址和worksheet工作表名字，可以给指定工作表下的所有有数据的单元格加上边框，如5图所示。

In [ ]:
from openpyxl.styles.borders import Border, Side　　
from openpyxl import Workbook      
def add_border(path,worksheet):   # 参数为excel文件地址和worksheet工作表
    
    # 定义边框类型
    
    
    
    # 加载excel文件
    
    # 打开指定工作表
    
    # 遍历所有单元格并判断其是否有数值，如果有数值就加上边框
    
    
    
    # 保存文件
    
    
    
    
    
    
    
    

## <font color='red'> 6.1 检测函数 </font>
请测试[test.xlsx](../../notebooks/generate-labels_and_re/data/test.xlsx)文件，看看是否能达到效果，或自行创建excel文件进行debug。

In [ ]:
from excel import clean_all   
clean_all('data/test.xlsx','label')  # 清理excel文件里的多余信息，方便学生debug

<font color='red'> 此处为参考答案，双击后查看 </font> <span class="graffiti-highlight graffiti-id_fqymn1h-id_o6q083h"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>
<p hidden>
def add_border1(path,workbook):
    wb = load_workbook(path)
    ws = wb[workbook]
    max_row = ws.max_row
    max_column = ws.max_column
    thin_border = Border(left=Side(style='thin'),
                   right=Side(style='thin'),
                   top=Side(style='thin'),
                   bottom=Side(style='thin'))
    for i in range(1,max_column +  1):
      for j in range(1,max_row + 1 ):
        if ws.cell(row=j,column=i).value:
          ws.cell(row=j, column=i).border = thin_border
    wb.save(path)
    
    
    
</p>

# <font color='royalblue'> 7. 批量修改文件
以上是处理单个订单的标签，对于多个订单的标签，我们可以用类似的方法一次完成。我们只需要将所有订单的Excel文件放入一个文件夹，然后就可以批量操作。

## 7.1 批量获取文件名  <span class="graffiti-highlight graffiti-id_js88i8k-id_fkjnqm7"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

需要引入`os`模块。该模块可自动获取文件路径及文件名。

- 1. 我们定义一个函数`file_name`，包含一个参数`file_dir`,该参数用于传入Excel文件所在的路径。随后定义一个列表`names`用于存储后续获取的Excel文件路径。`os.listdir`将返回指定的文件夹包含的文件及文件夹的名字的列表。

- 2. 我们只需要处理Excel文件，因此加入一个条件判断，只将带有“.xlsx”后缀的文件放入`names`列表。由于`os.listdir`只获取了文件名，但我们在写入数据时需要文件的完整路径，所以需要把文件所在的路径加在文件名前面，即`file_dir+file`。最后返回`names`列表，获得所有Excel文件的完整路径。

- 3. 然后就可以调用之前写好的读取信息及写入信息的函数，逐个将`names`列表中的路径传入函数，完成信息的读取和写入。

In [ ]:
import os
def file_name(file_dir):
    names=[]
    for file in os.listdir(file_dir):
        if ".xlsx" in file: # 只处理该路径下的Excel文件
            names.append(file_dir+file)
    return names
file_dir="data/"
pathess=file_name(file_dir)
pathess

## 7.2 运用函数批量操作文件

### <font color='red'>注意：</font>如果上方你的自定义函数出现错误，此处的代码将无法运行
如果你无法完成函数请将下面的函数名称换成`get_info1` 和 `write_info1` <span class="graffiti-highlight graffiti-id_3xwo8qs-id_rnbufy7"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

In [ ]:
%%time
from excel import get_info1
from excel import write_info1 


for path in pathess:
    info=get_info(path)
    write_info(path,info) 
print("Finished!")    

## 7.3 遍历不同的子文件夹 <span class="graffiti-highlight graffiti-id_pkhbgos-id_h4fro9t"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

因为不同的客户，我们的文件夹可能还包含子文件夹，那要一次获取母文件夹中的所有Excel文件，需要应用到`os.walk`函数。`os.walk`可遍历一个目录内各个子目录和子文件。它先遍历当前目录，返回三个值，分别是目录的路径，目录下子目录的名字，文件的名字。再遍历子目录，同样返回子目录的路径，子目录下的子目录的名字，子目录内的文件的名字。若还有子目录，则继续遍历，直到所有目录被遍历。因此需要三个变量`root, dirs, files`去接收它的返回值。

我们在`data/`路径下建了2个子目录，a和b，可以看到`os.walk`先遍历`data/`目录，再`data/a`,`data/b`，并找出所有目录下的文件。

In [ ]:
file_dir="data/"
for root, dirs, files in os.walk(file_dir):
    print(root,dirs,files,sep="\n******************\n")
    

In [ ]:
pathss=[] # 文件夹内所有文件（包括子目录）
file_dir="data/"
for root, dirs, files in os.walk(file_dir):
    path = [os.path.join(root, name) for name in files]
    pathss.extend(path)
pathss


```python 
path = [os.path.join(root, name) for name in files]```
这可以让程序变得非常简洁，也节省运算时间。通过看下面这个小例子就可以知道其原理：
```python
a=[i for i in range(10)]
a
>>[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]```
如果文件夹内还有除Excel之外的文件，需要将其从文件路径列表中剔除，然后再传入信息读取和写入的函数。

In [ ]:
result=[]
for i in pathss:
    if '.xlsx' in i:
        result.append(i)
result

# <font color='red'>8. 完成练习 </font> <span class="graffiti-highlight graffiti-id_sw79r0m-id_qfh7wnc"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

结合所用知识，完成下方步骤：
- 获取文件夹中所有excel文件
- 对每个excel文件，运用`get_info()`，`write_info()`，`add_border()`
- 查看结果是否正确

In [ ]:
# 将你的练习8内容写在这里





# <font color='royalblue'>9. 额外内容：正则表达式 </font> <span class="graffiti-highlight graffiti-id_qqfzs79-id_nsircax"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>
正则表达式（regular expression）又可以称作规则表达式，正则就是操作一大堆符串，就是用事先定义好的一些特定字符、及这些特定字符的组合，组成一个“规则字符串”，来匹配要检索的字符串。

In [ ]:
import re

## 9.1 原字符串 <span class="graffiti-highlight graffiti-id_sna70y9-id_kjcqma5"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>
原始字符串是Python中一类比较特殊的字符串，以大写字母R或者小写字母r开始。在原始字符串中，字符“\” 不再表示转义字符的含义。


In [ ]:
a = '\tabc'
print(a)

In [ ]:
a = r'\tabc'
print(a)

## 9.2 构造和使用正则表达式

### 9.2.1 大小写区分 <span class="graffiti-highlight graffiti-id_axcyv22-id_dyx5abr"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

从下方的代码我们可以看出，正则表达式是区分大小写的。它只匹配来第一行小写的abc，没有匹配第二行大写的ABC。

In [ ]:
text_to_search = '''
abcdefghijklmnopqurtuvwxyz
ABCDEFGHIJKLMNOPQRSTUVWXYZ
1234567890
Ha HaHa
MetaCharacters (Need to be escaped):
. ^ $ * + ? { } [ ] \ | ( )
coreyms.com
321-555-4321
123.555.1234
123*555*1234
800-555-1234
900-555-1234
Mr. Schafer
Mr Smith
Ms Davis
Mrs. Robinson
Mr. T
'''
pattern = re.compile(r'abc')
matches = pattern.finditer(text_to_search)
for match in matches:
    print(match)
    
    
    

In [ ]:
print(text_to_search[1:4])

### 9.2.2 万能匹配符号：. <span class="graffiti-highlight graffiti-id_pyj5web-id_y5en8jx"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

万能的匹配符号`.`它可以匹配所有的字符

In [ ]:
pattern = re.compile(r'.')
matches = pattern.finditer(text_to_search)
for match in matches:
    print(match)
    

    

那怎么只匹配`.`它自己呢，可以使用转义符号右斜杠：`\`

In [ ]:
pattern = re.compile(r'\.')
matches = pattern.finditer(text_to_search)
for match in matches:
    print(match)

### <font color='red'>9.2.3 练习 </font> 
仿照上面代码，匹配出下方字符串中的`coreyms.com`。

In [ ]:
import re

text = '''
321-555-4321
coreyms.com
123.555.1234
123*555*1234
800-555-1234
900-555-1234
Mr. Schafer
Mr Smith
Ms Davis
Mrs. Robinson
Mr. T
'''
pattern = re.compile(??)
matches = pattern.finditer(text_to_search)
for match in matches:
    print(match)


<font color='red'>答案 </font><span class="graffiti-highlight graffiti-id_loefq9m-id_ci2j6pa"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

## 9.3 转义字符: <span class="graffiti-highlight graffiti-id_0pzvif7-id_7y7xq22"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>
- .       - 任意字符除了新的一行 <br>
- \d      - 任意数字（0-9） <br>
- \D      - 不是数字 (0-9) <br>
- \w      - 任意字母和数字 (a-z, A-Z, 0-9, _) <br>
- \W      - 除了任意字母和数字 <br>
- \s      - 任意空格 (空格, tab, 新行) <br>
- \S      - 除了任意空格(space, tab, newline) <br>
- \b      - 单词边界 <br>
- \B      - 不是单词边界 <br>
- ^       - 字符串开头 <br>
- $       - 字符串结尾 <br>

### 9.3.1 基础转义字符 <span class="graffiti-highlight graffiti-id_x8v6iol-id_drdte5i"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

In [ ]:
text_to_search = '''
abcdefghijklmnopqurtuvwxyz
ABCDEFGHIJKLMNOPQRSTUVWXYZ
1234567890
Ha HaHa
MetaCharacters (Need to be escaped):
. ^ $ * + ? { } [ ] \ | ( )
coreyms.com
321-555-4321
123.555.1234
123*555*1234
800-555-1234
900-555-1234
Mr. Schafer
Mr Smith
Ms Davis
Mrs. Robinson
Mr. T
'''

让我们先试一下`\d`,`\D`,`\w`,`\W`

In [ ]:
import re
pattern = re.compile(r'\d\d')
matches = pattern.finditer(text_to_search)
for match in matches:
    print(match)

`\b`,`\B` <span class="graffiti-highlight graffiti-id_93ekpgl-id_gakbcyw"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

In [ ]:
pattern = re.compile(r'ey\B')
matches = pattern.finditer(text_to_search)
for match in matches:
    print(match)

再试试 `^`：字符串的开头 , `$`：字符串的结尾  <span class="graffiti-highlight graffiti-id_p0n0guj-id_zc90048"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

In [ ]:
sentence = 'Start a sentence and then bring it to the end'

pattern = re.compile(r'^Start')
matches = pattern.finditer(sentence)
for match in matches:
    print(match)
    


In [ ]:
pattern = re.compile(r'end$')
matches = pattern.finditer(sentence)
for match in matches:
    print(match)

### <font color='red'>9.3.2 练习 </font>

`使用转义字符`，每次只匹配出下方字符串中的:
- `Mr. Schafer`

`使用转义字符`，每次只匹配出下方字符串中的:
- `123.555.1234`

`使用转义字符`，一次性匹配下面所有字符串:
- `321-555-4321`
- `123.555.1234`
- `123*555*1234`
- `800-555-1234`
- `900-555-1234`

In [ ]:
text_to_search = '''
abcdefghijklmnopqurtuvwxyz
ABCDEFGHIJKLMNOPQRSTUVWXYZ
1234567890
Ha HaHa
MetaCharacters (Need to be escaped):
. ^ $ * + ? { } [ ] \ | ( )
coreyms.com
321-555-4321
123.555.1234
123*555*1234
800-555-1234
900-555-1234
Mr. Schafer
Mr Smith
Ms Davis
Mrs. Robinson
Mr. T
'''

In [ ]:
pattern = re.compile(???)
matches = pattern.finditer(text_to_search)
for match in matches:
    print(match)

<font color='red'>答案 </font><span class="graffiti-highlight graffiti-id_ylygek1-id_6eijv62"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

### 9.3.3 转义字符：字符组 [ ]  <span class="graffiti-highlight graffiti-id_rzfmlwr-id_lb88f87"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>
`[]`里囊括的所有字符，就是它能代表的所有选项。N选1。

例如下方我需要匹配`Hello`,`Hillo`,`Hullo`这三个单词。

In [ ]:
text = '''
Hello
Hillo
Hullo
goodluck
887-213-4512
Mr. Schafer
Mr Smith
Ms Davis
Mrs. Robinson
Mr. T
456
192
899
321
126
234
512
666
ABC
GHF
WWW
BFR
CDF
'''

pattern = re.compile(r'H[eiu]llo')
matches = pattern.finditer(text)
for match in matches:
    print(match)

字符组里的特殊符号：`-`<br> <span class="graffiti-highlight graffiti-id_oq33b2n-id_3rhm1kw"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>
- 如果我只想匹配上方3位的数字，开头数字范围为1-4又该如何做呢？
- 如果我只想匹配上方3个的字母，首字母范围是A到D又该怎么办呢？

In [ ]:
pattern = re.compile(r'[1-4]\d\d')
matches = pattern.finditer(text)
for match in matches:
    print(match)

In [ ]:
pattern = re.compile(r'[A-D]\w\w') # 换成小写也可
matches = pattern.finditer(text)
for match in matches:
    print(match)

字符组里的特殊符号：`^` <br> <span class="graffiti-highlight graffiti-id_z2j91fu-id_33o3jk8"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>
`^` 表示反义符号，加在组合里表示：所有不在该组的数值。
- 假如我想匹配首字符不为字母的所有下方3位字符串

In [ ]:
text = '''
A19
a83
123
444
*AS
)ow
wqs
'''

pattern = re.compile(r'[^a-zA-Z\s]\w\w')
matches = pattern.finditer(text)
for match in matches:
    print(match)

### <font color='red'>9.3.4 练习 </font>

像`321*555-4321`这10位数字，是电话号码格式的一种。请使用正则表达式，匹配出下方字符串中，所有以`900`和`800`开头的电话号码。

In [ ]:
text = '''
coreyms.com
321*555-4321
123.555.1234
123.555.1234
123*555*1234
900-555-1234
800-555-1234
900*555*1234
800-555*1234
900-555-1234
900-555-1234
Mr. Schafer
Mr Smith
Ms Davis
'''

pattern = re.compile(???)
matches = pattern.finditer(text)
for match in matches:
    print(match)

匹配下方字符串除了`bat`以外的所有单词。

In [ ]:
text = '''
cat
mat
pat
bat
'''
pattern = re.compile(???)
matches = pattern.finditer(text)
for match in matches:
    print(match)

<font color='red'>答案 </font><span class="graffiti-highlight graffiti-id_8sh1ni1-id_z7i88de"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

### 9.3.5 转义字符：{ }, * , +, ? ,()   <span class="graffiti-highlight graffiti-id_9iw9mr9-id_04ydx5d"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

- `*` : 0次或者更多 <br>
- `+` :    1次或者更多  <br>
- `?` :    1次或者0次  <br>
- `{3}` :  确切的次数  <br>
- `{3,4}` :  范围 (Minimum, Maximum) <br>
- `()`:组合

`{ }`：可以指定匹配确切的次数。就能防止过多的代码和误写。 <span class="graffiti-highlight graffiti-id_cyuochz-id_wz3uhup"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

In [ ]:
text = '''
coreyms.com
321*555-4321
123.555.1234
123.555.1234
123*555*1234
900-555-1234
800-555-1234
900*555*1234
800-555*1234
900-555-1234
900-555-1234
'''
pattern = re.compile(r'\d\d\d[.*-]\d\d\d[.*-]\d\d\d\d')
matches = pattern.finditer(text)
for match in matches:
    print(match)

可以看见上方是我们之前学习的方法，非常的浪费时间还容易写错。现在我们可以使用`{}` 来指定匹配次数

In [ ]:
pattern = re.compile(r'\d{3}[.*-]\d{3}[.*-]\d{4}')
matches = pattern.finditer(text)
for match in matches:
    print(match)

<span class="graffiti-highlight graffiti-id_bjes0or-id_x9lnl49"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>
- `*` : 0次或者更多 
- `+` :    1次或者更多  <br>
- `?` :    1次或者0次  <br>
这次我们使用这三个符号来匹配名字
- 如果我们只想匹配Mr. Mr 开头的字符，也就是先生，应该怎么写呢？

In [ ]:
text = """
Mr. Schafer
Mr Smith
Ms Davis
Mrs. Robinson
Mr. T
"""

pattern = re.compile(r'Mr\.?\s[A-Z]\w*')
matches = pattern.finditer(text)
for match in matches:
    print(match)

<span class="graffiti-highlight graffiti-id_ccp6rke-id_qx0akog"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>
- 如果我们只想匹配Mrs. Ms 开头的字符，也就是女士，应该怎么写呢？<br>
`( )`:代表组合，里面可以配合`|`字符使用，代表或者。

In [ ]:
pattern = re.compile(r'M(rs|s)\.?\s[A-Z]\w*')
matches = pattern.finditer(text)
for match in matches:
    print(match)
    
    

`()`组合的使用 <span class="graffiti-highlight graffiti-id_c7m2vty-id_58inxbv"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

In [ ]:
urls = '''
https://www.baidu.com
http://sina.cn
https://www.nasa.gov
https://tsinghua.edu
'''

pattern = re.compile(r'https?://(www\.)?(\w+)(\.\w+)')
matches = pattern.finditer(urls)
for match in matches:
    print(match)

### <font color='red'>9.3.4 练习 </font>
使用正则表达式，匹配下面字符串的所有名字。

In [ ]:
text = """
Mr. Schafer
Mr Smith
Ms Davis
Mrs. Robinson
Mr. T
"""
pattern = re.compile(???)
matches = pattern.finditer(text)
for match in matches:
    print(match)

使用正则表达式，匹配下面所有电子邮件地址

In [ ]:
emails = """
CoreyMSchafer@qq.com
corey.schafer@university.edu
corey-321-schafer@new-land.cn
"""

pattern = re.compile(???)  
matches = pattern.finditer(emails)
for match in matches:
    print(match)
    
    

<font color='red'>答案 </font><span class="graffiti-highlight graffiti-id_x3p6ek7-id_5mpuh6d"><i></i><button><img src="https://gitee.com/wenjian89/ai-edu/raw/master/icon-loudspeaker.jpeg" alt="点击查看解析" style="width: 20px; " align="right"/></button></span>

### <font>9.3.5 搜索匹配文本和替换 </font>
`pattern.search()`: 返回第一次的匹配

In [ ]:
import re
text = """
Cat
cat
mat
pat
bat
"""

pattern = re.compile(r'[cmp]at')  
matches = pattern.search(text)
print(matches)



配合上`group()`就能返回匹配的文本信息列

In [ ]:
print(matches.group())

使用`re.IGNORECASE`可以使正则表达式省略大小写

In [ ]:
pattern = re.compile(r'[cmp]at',re.IGNORECASE)  
matches = pattern.search(text)
print(matches.group())

`replace(from,to)` ：替换文本

In [ ]:
pattern = re.compile(r'[cmp]at',re.IGNORECASE)  
matches = pattern.search(text)
new_text = matches.group().replace('C','?')
print(new_text)

# <font color='red'>10. 综合练习</font>

目标：提取excel文件`purchase.xlsx`的`D列`备注信息中的：采购号。<br>格式为`EPR-XX-XXXXX`。<br>
`XX`：两位数字<br>
`XXXXX`:最后一个部分描述为，第一个字母一般为大写x或者省略，后面跟着编号数字，3位数字或者4位数字。

请从云端下载excel文件，[purchase.xlsx](../../notebooks/generate-labels_and_re/data/purchase.xlsx),观察备注栏中的信息，编写正则表达式来匹配信息，将提取到的采购号，填入对应`E列`的`PR编号`中

<img style="float: left;" width = "850" height = "300" src=./images/13.png> 

### 你可以根据下面的步骤来完成该练习：
- 1.编写函数，用于清理e列从第二行开始的所有信息，方便之后debug，和观察变化。
- 2.建立一个空列表，用来储存D列的所有信息。
- 3.将D列的所有信息加入空列表中。 <br>
- 对列表中的数据：
- 4.创建一个新列表
- 5.数据统一规范化：EPR编号的字符串是带有问号“?”的，比如“EPR-14?X1059”，因此需要先将其替换成我们想要的“-”。可以使用replace()
- 6.编写正则表达式来匹配列表当中的每个数据，提取出符合正则表达式的字符串并加入刚创建的新列表中
- 7.将列表中的处理和匹配完的数据写入E列当中


In [ ]:
### 完成你的练习

 <p hidden>
def clean_column_e(path,worksheet):
    wb = load_workbook(path)
    ws = wb[worksheet]
    max_row = ws.max_row
    max_column = ws.max_column
    for j in range(2,max_row + 1 ):
        ws.cell(row=j, column=5).value = None
    wb.save(path)
</p>

<p hidden>"""
max_row = ws.max_row
max_column = ws.max_column
column_d = []
for i in range (2, max_row+1):
    value = ws.cell(row=i,column=4).value
    value = value.replace("?","-")
    column_d.append(value)
</p> <p hidden>
processed_data = []
pattern = re.compile(r'EPR-\d{2}-X?\d{3}(\d)?',re.IGNORECASE)
for i in range(len(column_d)):
   data = column_d[i]
   match = pattern.search(data).group()
   processed_data.append(match)
</p><p hidden>
for i in range(len(processed_data)):
   ws.cell(row=i+2,column=5).value = processed_data[i]
wb.save(path)
</p>